In [4]:
import os
import glob
from multiprocessing.pool import Pool

import numpy as np
from utils_de import imread, imwrite
from PIL import Image

import random
import math
import torchvision.transforms.functional as F
import numpy as np
import cv2
import json
from PIL import Image
from utils_de import transform
import matplotlib.pyplot as plt

In [152]:
## dark image degradation
# input_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/training/images'
# mask_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/training/mask'
# output_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/degra/dark/training'

input_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/test/images'
mask_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/test/mask'
output_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/degra/dark/test'

brightness_factor = 0.3

input_path = os.listdir(input_dir)
for filename in input_path:
    f_img = os.path.join(input_dir, filename)
    name = os.path.splitext(filename)[0]
    f_mask = os.path.join(mask_dir, name + '_mask.gif')
    img = Image.open(f_img).convert('RGB')
    mask = Image.open(f_mask).convert('L')
    img = img.resize((512,512))
    mask = mask.resize((512,512))
    img = F.adjust_brightness(img, brightness_factor)
    img = np.array(img)
    mask = np.repeat(np.expand_dims(np.array(mask),2), 3, axis=2)/255.0
    img = img * mask
    img = img.astype("uint8")
    img = Image.fromarray(img, mode='RGB')
    img.save(os.path.join(output_dir, filename))

In [167]:
## border bright image degradation
input_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/training/images'
mask_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/training/mask'
output_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/degra/border_bright/training'
# input_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/test/images'
# mask_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/test/mask'
# output_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/degra/border_bright/test'

# 超参数
weight_r = [251/255,141/255,177/255]
weight_g = [249/255,238/255,195/255]
weight_b = [246/255,238/255,147/255]

num = 0
w = 512 
h = 512

input_path = os.listdir(input_dir)
for filename in input_path:
    f_img = os.path.join(input_dir, filename)
    name = os.path.splitext(filename)[0]
    f_mask = os.path.join(mask_dir, name + '_mask.gif')
    img = Image.open(f_img).convert('RGB')
    mask = Image.open(f_mask).convert('L')
    img = img.resize((512,512))
    img = np.array(img) / 255.0
    mask = mask.resize((512,512))
    mask = np.repeat(np.expand_dims(np.array(mask),2), 3, axis=2)/255.0

    # 中心四分之一处随机center
    w0_a = random.randint(w/2-int(w/8),w/2+int(w/8))
    h0_a = random.randint(h/2-int(h/8),h/2+int(h/8))
    center_a = [w0_a, h0_a]
    # 接近于1的diameter
    wei_dia_a =0.7 + (1.0-0.7) * random.random()
    dia_a = min(h,w)*wei_dia_a
    Y_a, X_a = np.ogrid[:h, :w]
    dist_from_center_a = np.sqrt((X_a - center_a[0]) ** 2 + (Y_a - center_a[1]) ** 2)
    circle_a = dist_from_center_a <= (int(dia_a / 2))
    # mask
    mask_a = np.zeros((h, w))
    mask_a[~circle_a] = np.mean(img) #np.multiply(A[0], (1 - t))

    # gaussianblur
    delta_circle = mask_a * 2
    dia = max(center_a[0],h-center_a[0],center_a[1],h-center_a[1])*2
    gauss_rad = int(np.abs(dia-dia_a))
    sigma = 2/3*gauss_rad
    if(gauss_rad % 2) == 0:
        gauss_rad= gauss_rad+1
    delta_circle = cv2.GaussianBlur(delta_circle, (gauss_rad, gauss_rad), sigma)

    # add
    delta_circle = np.transpose(np.array([weight_r[num]*delta_circle,weight_g[num]*delta_circle,weight_b[num]*delta_circle]), (1,2,0))
    img = img + delta_circle
    img = np.maximum(img, 0)
    img = np.minimum(img, 1)

    # save
    img = img * 255.0
    img = img * mask
    img = img.astype("uint8")
    img = Image.fromarray(img, mode='RGB')
    img.save(os.path.join(output_dir, filename))

In [180]:
## center dark degradation
input_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/test/images'
mask_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/test/mask'
output_dir = '/home/yangby/yangby/stable-diffusion-main/datasets/DIRVE/degra/center_dark/test'

# 超参数
w = 512 
h = 512
brightness_factor = -0.4

input_path = os.listdir(input_dir)
for filename in input_path:
    f_img = os.path.join(input_dir, filename)
    name = os.path.splitext(filename)[0]
    f_mask = os.path.join(mask_dir, name + '_mask.gif')
    img = Image.open(f_img).convert('RGB')
    mask = Image.open(f_mask).convert('L')
    img = img.resize((512,512))
    img = np.array(img) / 255.0
    mask = mask.resize((512,512))
    region_mask = np.repeat(np.expand_dims(np.array(mask),2), 3, axis=2)/255.0

    # diameter
    diameter_circle = random.randint(int(0.4 * w), int(0.7 * w))
    # 中心四分之一处center
    center =[random.randint(w/4,w*3/4),random.randint(h*3/8,h*5/8)]
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0]) ** 2 + (Y - center[1]) ** 2)
    circle = dist_from_center <= (int(diameter_circle/2))

    mask = np.zeros((h, w))
    mask[circle] = 1

    mask = mask * brightness_factor

    rad_w = random.randint(int(diameter_circle*0.55), int(diameter_circle*0.75))
    rad_h = random.randint(int(diameter_circle*0.55), int(diameter_circle*0.75))
    sigma = 2/3 * max(rad_h, rad_w)*1.2

    if (rad_w % 2) == 0: rad_w = rad_w + 1
    if(rad_h % 2) ==0 : rad_h =rad_h + 1

    mask = cv2.GaussianBlur(mask, (rad_w, rad_h), sigma)
    mask = np.transpose(np.array([mask, mask, mask]), (1,2,0))
    img = img + mask
    img = img * region_mask
    img = np.maximum(img, 0)
    img = np.minimum(img, 1)

    img = img * 255.0
    img = img.astype("uint8")
    img = Image.fromarray(img, mode='RGB')
    img.save(os.path.join(output_dir, filename))
